In [24]:
pip install ngsolve

Note: you may need to restart the kernel to use updated packages.


In [46]:
from ngsolve import *
from ngsolve.webgui import Draw
import numpy as np
from netgen.geom2d import SplineGeometry
import matplotlib.tri as mtri
import matplotlib.pyplot as plt
from pytictoc import TicToc
import ezyrb
from ezyrb import POD, RBF, Database, GPR
from ezyrb import ReducedOrderModel as ROM
%matplotlib inline
from sklearn.gaussian_process import kernels

In [47]:
# viscosity
nu = 0.01

# timestepping parameters
tau = 0.01
tend = 5


In [48]:
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet"))
geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl", maxh=0.02)
mesh = Mesh( geo.GenerateMesh(maxh=0.08))
mesh.Curve(3)
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

BaseWebGuiScene

In [50]:
#Construct function space

V = VectorH1(mesh,order=3, dirichlet="wall|cyl|inlet")
Q = H1(mesh,order=2)
X = V*Q
u,p = X.TrialFunction()
v,q = X.TestFunction()

In [51]:
stokes = nu*InnerProduct(grad(u), grad(v))+div(u)*q+div(v)*p - 1e-10*p*q
a = BilinearForm(X, symmetric=True)
a += stokes*dx
a.Assemble()

# nothing here ...
f = LinearForm(X)   
f.Assemble()

# gridfunction for the solution
gfu = GridFunction(X)

# parabolic inflow at inlet:
uin = CoefficientFunction( (1.5*4*y*(0.41-y)/(0.41*0.41), 0) )
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

# solve Stokes problem for initial conditions:
inv_stokes = a.mat.Inverse(X.FreeDofs())

res = f.vec.CreateVector()
res.data = f.vec - a.mat*gfu.vec
gfu.vec.data += inv_stokes * res


# matrix for implicit Euler 
mstar = BilinearForm(X, symmetric=True)
mstar += (u*v + tau*stokes)*dx
mstar.Assemble()
inv = mstar.mat.Inverse(X.FreeDofs(), inverse="sparsecholesky")

# the non-linear term 
conv = BilinearForm(X, nonassemble = True)
conv += (grad(u) * u) * v * dx

# for visualization
Draw (Norm(gfu.components[0]), mesh, "velocity")


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

BaseWebGuiScene

In [52]:
t1 = TicToc() #create instance of class
t1.tic() #Start timer1
# implicit Euler/explicit Euler splitting method:
t = 0
snapshots = []
param = []
velocity = CoefficientFunction(gfu.components[0:2])

with TaskManager():
    while t < tend:
        print ("t=", t, end="\r")
        param.append(t)

        conv.Apply (gfu.vec, res)
        res.data += a.mat*gfu.vec
        gfu.vec.data -= tau * inv * res    
        snapshots.append(gfu.vec)
        t = t + tau
Draw(velocity, mesh, "vel")
t1.toc()        

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

Elapsed time is 1.231912 seconds.


In [39]:
snapshots = np.array(snapshots)
param = np.array(param).reshape(-1,1)
db = Database(param, snapshots)
print(np.shape(db.snapshots))
print(np.shape(db.parameters))

(501, 4550)
(501, 1)


In [40]:
pod = POD('svd', rank = 5, save_memory = True)


In [41]:
#Creates an instance of GPR/RBF
# provide parameters like kernel, optimize, etc. 
gpr = GPR()

In [42]:
rom = ROM(db, pod, gpr) # pass the instances and not the classes

In [43]:
rom.fit()

In [44]:
# computes u_red = U U^T S, where U = pod modes, S = snapshot matrix
#pass the new parameter where you want to compute the reduced solutions
t1.tic() #Start timer1

t = 0
while t < tend:
  t = t + tau
  pred_sol = rom.predict(t)
  gfu1 = GridFunction(X)
  gfu1.vec.FV().NumPy()[:] = pred_sol
  velocity = CoefficientFunction(gfu1.components[0:2])
t1.toc() #Start timer1
Draw(velocity, mesh, "vel")


Elapsed time is 0.237995 seconds.


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

BaseWebGuiScene

In [45]:
# Compute the mean norm of the relative error vectors of predicted test snapshots
err = rom.test_error(db)
print(err)

4.776142232676212e-16
